# Speech Understanding | Programming Assignment 2

    Ayush Abrol B20AI052

---

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os
import wandb
import random
import math
from tqdm import tqdm
from pathlib import Path
from typing import List, Tuple, Union

from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from scipy.interpolate import interp1d
from scipy.optimize import brentq
import fire

import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.functional import pad
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
from torch import Tensor
from torchaudio._internal import download_url_to_file
from torchaudio.transforms import Resample
import torchaudio.transforms as trans
from torchaudio.datasets.utils import _extract_zip, _load_waveform
import warnings
warnings.filterwarnings("ignore")

### Question 1

**Goal**: In speaker verification, the training dataset consists of audio clips paired with speaker IDs, denoted as (D = (xi, yi)). Given an audio clip (x) and a reference clip (x0), the objective is to ascertain whether (x0) and (x) belong to the same speaker. 

**Tasks**: 

    1. Choose three pre-trained models from the list: ’ecapa tdnn’, ’hubert large’, ’wav2vec2 xlsr’, ’unispeech sat’, ’wavlm base plus’, ’wavlm large’ trained on the VoxCeleb1 dataset. You can find the pre-trained models on this link.
    
    2. —- Calculate the EER(%) on the VoxCeleb1-H dataset using the above selected models. You can get the 
    dataset from here.
    
    3. Compare your result with Table II of the WavLM paper. 

    4. Evaluate the selected models on the test set of any one Indian language of the Kathbath Dataset. Report the EER(%). 
    
    5. Fine-tune, the best model on the validation set of the selected language of Kathbath Dataset. Report the EER(%). 
    
    6. Provide an analysis of the results along with plausible reasons for the observed outcomes. 

In [2]:
def calculate_eer_percentage(labels, preds):
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

In [3]:
class Res2Conv1dReluBn(nn.Module):

    def __init__(self, channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=True, scale=4):
        super().__init__()
        assert channels % scale == 0, "{} % {} != 0".format(channels, scale)
        self.scale = scale
        self.width = channels // scale
        self.nums = scale if scale == 1 else scale - 1

        self.convs = []
        self.bns = []
        for i in range(self.nums):
            self.convs.append(nn.Conv1d(self.width, self.width, kernel_size, stride, padding, dilation, bias=bias))
            self.bns.append(nn.BatchNorm1d(self.width))
        self.convs = nn.ModuleList(self.convs)
        self.bns = nn.ModuleList(self.bns)

    def forward(self, x):
        out = []
        spx = torch.split(x, self.width, 1)
        for i in range(self.nums):
            if i == 0:
                sp = spx[i]
            else:
                sp = sp + spx[i]
            # Order: conv -> relu -> bn
            sp = self.convs[i](sp)
            sp = self.bns[i](F.relu(sp))
            out.append(sp)
        if self.scale != 1:
            out.append(spx[self.nums])
        out = torch.cat(out, dim=1)

        return out


class Conv1dReluBn(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=True):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=bias)
        self.bn = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        return self.bn(F.relu(self.conv(x)))


class SE_Connect(nn.Module):
    def __init__(self, channels, se_bottleneck_dim=128):
        super().__init__()
        self.linear1 = nn.Linear(channels, se_bottleneck_dim)
        self.linear2 = nn.Linear(se_bottleneck_dim, channels)

    def forward(self, x):
        out = x.mean(dim=2)
        out = F.relu(self.linear1(out))
        out = torch.sigmoid(self.linear2(out))
        out = x * out.unsqueeze(2)

        return out


class SE_Res2Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, scale, se_bottleneck_dim):
        super().__init__()
        self.Conv1dReluBn1 = Conv1dReluBn(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.Res2Conv1dReluBn = Res2Conv1dReluBn(out_channels, kernel_size, stride, padding, dilation, scale=scale)
        self.Conv1dReluBn2 = Conv1dReluBn(out_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.SE_Connect = SE_Connect(out_channels, se_bottleneck_dim)

        self.shortcut = None
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1,
            )

    def forward(self, x):
        residual = x
        if self.shortcut:
            residual = self.shortcut(x)

        x = self.Conv1dReluBn1(x)
        x = self.Res2Conv1dReluBn(x)
        x = self.Conv1dReluBn2(x)
        x = self.SE_Connect(x)

        return x + residual

class AttentiveStatsPool(nn.Module):
    def __init__(self, in_dim, attention_channels=128, global_context_att=False):
        super().__init__()
        self.global_context_att = global_context_att

        # Use Conv1d with stride == 1 rather than Linear, then we don't need to transpose inputs.
        if global_context_att:
            self.linear1 = nn.Conv1d(in_dim * 3, attention_channels, kernel_size=1)  # equals W and b in the paper
        else:
            self.linear1 = nn.Conv1d(in_dim, attention_channels, kernel_size=1)  # equals W and b in the paper
        self.linear2 = nn.Conv1d(attention_channels, in_dim, kernel_size=1)  # equals V and k in the paper

    def forward(self, x):

        if self.global_context_att:
            context_mean = torch.mean(x, dim=-1, keepdim=True).expand_as(x)
            context_std = torch.sqrt(torch.var(x, dim=-1, keepdim=True) + 1e-10).expand_as(x)
            x_in = torch.cat((x, context_mean, context_std), dim=1)
        else:
            x_in = x

        # DON'T use ReLU here! In experiments, I find ReLU hard to converge.
        alpha = torch.tanh(self.linear1(x_in))
        # alpha = F.relu(self.linear1(x_in))
        alpha = torch.softmax(self.linear2(alpha), dim=2)
        mean = torch.sum(alpha * x, dim=2)
        residuals = torch.sum(alpha * (x ** 2), dim=2) - mean ** 2
        std = torch.sqrt(residuals.clamp(min=1e-9))
        return torch.cat([mean, std], dim=1)


class ECAPA_TDNN(nn.Module):
    def __init__(self, feat_dim=80, channels=512, emb_dim=192, global_context_att=False,
                 feat_type='fbank', sr=16000, feature_selection="hidden_states", update_extract=False, config_path=None):
        super().__init__()

        self.feat_type = feat_type
        self.feature_selection = feature_selection
        self.update_extract = update_extract
        self.sr = sr

        if feat_type == "fbank" or feat_type == "mfcc":
            self.update_extract = False

        win_len = int(sr * 0.025)
        hop_len = int(sr * 0.01)

        if feat_type == 'fbank':
            self.feature_extract = trans.MelSpectrogram(sample_rate=sr, n_fft=512, win_length=win_len,
                                                        hop_length=hop_len, f_min=0.0, f_max=sr // 2,
                                                        pad=0, n_mels=feat_dim)
        elif feat_type == 'mfcc':
            melkwargs = {
                'n_fft': 512,
                'win_length': win_len,
                'hop_length': hop_len,
                'f_min': 0.0,
                'f_max': sr // 2,
                'pad': 0
            }
            self.feature_extract = trans.MFCC(sample_rate=sr, n_mfcc=feat_dim, log_mels=False,
                                              melkwargs=melkwargs)
        else:
            
            self.feature_extract = torch.hub.load('s3prl/s3prl', feat_type)
            
            if len(self.feature_extract.model.encoder.layers) == 24 and hasattr(self.feature_extract.model.encoder.layers[23].self_attn, "fp32_attention"):
                self.feature_extract.model.encoder.layers[23].self_attn.fp32_attention = False
            if len(self.feature_extract.model.encoder.layers) == 24 and hasattr(self.feature_extract.model.encoder.layers[11].self_attn, "fp32_attention"):
                self.feature_extract.model.encoder.layers[11].self_attn.fp32_attention = False

            self.feat_num = self.get_feat_num()
            self.feature_weight = nn.Parameter(torch.zeros(self.feat_num))

        if feat_type != 'fbank' and feat_type != 'mfcc':
            freeze_list = ['final_proj', 'label_embs_concat', 'mask_emb', 'project_q', 'quantizer']
            for name, param in self.feature_extract.named_parameters():
                for freeze_val in freeze_list:
                    if freeze_val in name:
                        param.requires_grad = False
                        break

        if not self.update_extract:
            for param in self.feature_extract.parameters():
                param.requires_grad = False

        self.instance_norm = nn.InstanceNorm1d(feat_dim)
        # self.channels = [channels] * 4 + [channels * 3]
        self.channels = [channels] * 4 + [1536]

        self.layer1 = Conv1dReluBn(feat_dim, self.channels[0], kernel_size=5, padding=2)
        self.layer2 = SE_Res2Block(self.channels[0], self.channels[1], kernel_size=3, stride=1, padding=2, dilation=2, scale=8, se_bottleneck_dim=128)
        self.layer3 = SE_Res2Block(self.channels[1], self.channels[2], kernel_size=3, stride=1, padding=3, dilation=3, scale=8, se_bottleneck_dim=128)
        self.layer4 = SE_Res2Block(self.channels[2], self.channels[3], kernel_size=3, stride=1, padding=4, dilation=4, scale=8, se_bottleneck_dim=128)

        # self.conv = nn.Conv1d(self.channels[-1], self.channels[-1], kernel_size=1)
        cat_channels = channels * 3
        self.conv = nn.Conv1d(cat_channels, self.channels[-1], kernel_size=1)
        self.pooling = AttentiveStatsPool(self.channels[-1], attention_channels=128, global_context_att=global_context_att)
        self.bn = nn.BatchNorm1d(self.channels[-1] * 2)
        self.linear = nn.Linear(self.channels[-1] * 2, emb_dim)


    def get_feat_num(self):
        self.feature_extract.eval()
        wav = [torch.randn(self.sr).to(next(self.feature_extract.parameters()).device)]
        with torch.no_grad():
            features = self.feature_extract(wav)
        select_feature = features[self.feature_selection]
        if isinstance(select_feature, (list, tuple)):
            return len(select_feature)
        else:
            return 1

    def get_feat(self, x):
        if self.update_extract:
            x = self.feature_extract([sample for sample in x])
        else:
            with torch.no_grad():
                if self.feat_type == 'fbank' or self.feat_type == 'mfcc':
                    x = self.feature_extract(x) + 1e-6  # B x feat_dim x time_len
                else:
                    x = self.feature_extract([sample for sample in x])

        if self.feat_type == 'fbank':
            x = x.log()

        if self.feat_type != "fbank" and self.feat_type != "mfcc":
            x = x[self.feature_selection]
            if isinstance(x, (list, tuple)):
                x = torch.stack(x, dim=0)
            else:
                x = x.unsqueeze(0)
            norm_weights = F.softmax(self.feature_weight, dim=-1).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
            x = (norm_weights * x).sum(dim=0)
            x = torch.transpose(x, 1, 2) + 1e-6

        x = self.instance_norm(x)
        return x

    def forward(self, x):
        x = self.get_feat(x)

        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)

        out = torch.cat([out2, out3, out4], dim=1)
        out = F.relu(self.conv(out))
        out = self.bn(self.pooling(out))
        out = self.linear(out)

        return out


def ECAPA_TDNN_SMALL(feat_dim, emb_dim=256, feat_type='fbank', sr=16000, feature_selection="hidden_states", update_extract=False, config_path=None):
    return ECAPA_TDNN(feat_dim=feat_dim, channels=512, emb_dim=emb_dim,
                      feat_type=feat_type, sr=sr, feature_selection=feature_selection, update_extract=update_extract, config_path=config_path)

In [4]:
def init_model(model_name, checkpoint=None):
    if model_name == 'unispeech_sat':
        config_path = 'models/unispeech_sat.th'
        model = ECAPA_TDNN_SMALL(feat_dim=1024, feat_type='unispeech_sat', config_path=config_path)
    elif model_name == 'wavlm_base_plus':
        config_path = None
        model = ECAPA_TDNN_SMALL(feat_dim=768, feat_type='wavlm_base_plus', config_path=config_path)
    elif model_name == 'wavlm_large':
        config_path = None
        model = ECAPA_TDNN_SMALL(feat_dim=1024, feat_type='wavlm_large', config_path=config_path)
    elif model_name == 'hubert_large':
        config_path = None
        model = ECAPA_TDNN_SMALL(feat_dim=1024, feat_type='hubert_large_ll60k', config_path=config_path)
    elif model_name == 'wav2vec2_xlsr':
        config_path = None
        model = ECAPA_TDNN_SMALL(feat_dim=1024, feat_type='wav2vec2_xlsr', config_path=config_path)
    else:
        model = ECAPA_TDNN_SMALL(feat_dim=40, feat_type='fbank')

    if checkpoint is not None:
        state_dict = torch.load(checkpoint, map_location=lambda storage, loc: storage)
        model.load_state_dict(state_dict['model'], strict=False)
    return model


In [5]:
def verification(model,  wav1, wav2, use_gpu=True, checkpoint=None):
    resample1 = Resample(orig_freq=16000, new_freq=16000)
    resample2 = Resample(orig_freq=16000, new_freq=16000)
    wav1 = resample1(wav1)
    wav2 = resample2(wav2)

    if use_gpu:
        model = model.cuda()
        wav1 = wav1.cuda()
        wav2 = wav2.cuda()
        
    model.eval()
    with torch.no_grad():
        emb1 = model(wav1)
        emb2 = model(wav2)

    sim = F.cosine_similarity(emb1, emb2)
    print("The similarity score between two audios is {:.4f} (-1.0, 1.0).".format(sim[0].item()))
    return sim

### VoXCeleb1 dataset

In [6]:
class AudioDatasetVox(Dataset):
    def __init__(self, root):

        self.root = root
        self.src1, self.src2, self.labels = self.read_data()

    def read_data(self):
        src1 = []
        src2 = []
        labels = []

        datafile = open(self.root, 'r')
        for dataline in datafile:
            dataline = dataline.strip().split()
            src1.append(dataline[1])
            src2.append(dataline[2])
            labels.append(dataline[0])
        
        return src1, src2, labels
    
    def __getitem__(self, idx):
        src1, src1_samplingrate = torchaudio.load(os.path.join("/root/voxceleb1/wav/", self.src1[idx]))
        src2, src2_samplingrate = torchaudio.load(os.path.join("/root/voxceleb1/wav/", self.src2[idx]))

        if src1.size(1) < 16000:
            src1 = pad(src1, (0, 16000 - src1.size(1)))
        else:
            src1 = src1[:, :16000]
        
        if src2.size(1) < 16000:
            src2 = pad(src2, (0, 16000 - src2.size(1)))
        else:
            src2 = src2[:, :16000]
        
        return src1[0], src2[0], self.labels[idx]

In [7]:
#initialise dataloader
dataset = AudioDatasetVox(root="root/VoxCelebVerification.txt")
dataloader = DataLoader(dataset, batch_size=2)

In [ ]:
#initialise model
model = init_model("wavlm_base_plus", checkpoint="checkpoints/wavlm_base_plus.pt")
model.eval()

#calculate eer for each model
eer_batchwise = []

for i, data in enumerate(dataloader):

    src1, src2, label = data[0], data[1], data[2]

    src1 = src1
    src2 = src2

    similarity_scores = verification(model, src1, src2)
    print(similarity_scores)
    similarity_scores = (similarity_scores >= 0.0).float()
    predictions = similarity_scores

    eer_score = calculate_eer_percentage(label, predictions)
    print("[Batch {}] EER score: {}".format(i, eer_score))
    eer_batchwise.append(eer_score)

print("EER Score:", sum(eer_batchwise)/len(eer_batchwise))

In [10]:
class AudioDatasetKathBath(Dataset):
    def __init__(self, root):

        self.root = root
        self.src1, self.src2, self.labels = self.read_data()
        self.read_hindidata()

    def read_data(self):
        src1 = []
        src2 = []
        labels = []

        file = open(self.root, 'r')
        for line in file:
            line = line.strip().split()
            src1.append(line[1])
            src2.append(line[2])
            labels.append(int(line[0]))
        
        return src1, src2, labels
    
    def read_hindidata(self):

        for i in range(len(self.src1)):
            
            split_path = self.src1[i].split('hindi')[1]
            split_path = split_path.split('/')
            
            new_path = os.path.join("test_known",split_path[2],split_path[4])
            new_path = os.path.join("root/KathBath",new_path)
            
            new_path = new_path.replace('.wav', '.m4a')
            
            self.src1[i] = new_path

        for i in range(len(self.src2)):
            
            split_path = self.src2[i].split('hindi')[1]
            split_path = split_path.split('/')
            
            new_path = os.path.join("test_known",split_path[2],split_path[4])
            new_path = os.path.join("root/KathBath",new_path)
            
            new_path = new_path.replace('.wav', '.m4a')
            
            self.src2[i] = new_path
    
    def __getitem__(self, idx):
        src1, src1_samplingrate = librosa.load(self.src1[idx])
        src2, src2_samplingrate = librosa.load(self.src2[idx])

        src1 = torch.tensor(src1, dtype=torch.float32)
        src1 = src1.unsqueeze(0)
        src2 = torch.tensor(src2, dtype=torch.float32)
        src2 = src2.unsqueeze(0)

        if src1.size(1) < 16000:
            src1 = pad(src1, (0, 16000 - src1.size(1)))
        else:
            src1 = src1[:, :16000]
        
        if src2.size(1) < 16000:
            src2 = pad(src2, (0, 16000 - src2.size(1)))
        else:
            src2 = src2[:, :16000]
        
        return src1[0], src2[0], self.labels[idx]

In [13]:
#initialise dataloader
dataset = AudioDatasetKathBath(root="root/KathBath_Metadata/hindi/test_known_data.txt")
dataloader = DataLoader(dataset, batch_size=2)

In [ ]:
#initialise model
model = init_model("wavlm_base_plus", checkpoint="checkpoints/WavLM-Base+.pt")
model.eval()

#calculate eer values
eer_batchwise = []

for i, data in enumerate(dataloader):

    src1, src2, label = data[0], data[1], data[2]

    src1 = src1
    src2 = src2

    similarity_scores = verification(model, src1, src2)
    print(similarity_scores)
    similarity_scores = (similarity_scores >= 0.0).float()
    predictions = similarity_scores

    eer_score = calculate_eer_percentage(label, predictions)
    print("[Batch {}] EER score: {}".format(i, eer_score))
    eer_batchwise.append(eer_score)

print("EER Score:", sum(eer_batchwise)/len(eer_batchwise))

In [15]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Gradio for Question 1

In [19]:
import gradio as gr

def verify_audio_model1(model, audio1, audio2):
    # Verify similarity between two audio files using model 1
    audio1_data = audio1[1] / np.max(np.abs(audio1[1]))
    audio2_data = audio2[1] / np.max(np.abs(audio2[1]))
    audio1_tensor = torch.tensor(audio1_data, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    audio2_tensor = torch.tensor(audio2_data, dtype=torch.float32).unsqueeze(0).to(DEVICE)
    similarity = verification(model, audio1_tensor, audio2_tensor)
    return str(similarity.cpu().detach().numpy()[0])

In [ ]:
#initialise model
model = init_model("wavlm_base_plus", checkpoint="checkpoints/wavelmbase+/WavLM-Base+.pt").to(device)
model.eval()

# Gradio interface for audio verification using model 1
audio1_input = gr.components.Audio(source="upload", label="Upload test sample 1", type="numpy")
audio2_input = gr.components.Audio(source="upload", label="Upload test sample 2", type="numpy")
similarity_output = gr.outputs.Label(label="s=Simimlarity Score")
gr.Interface(fn=verify_audio_model1, inputs=[audio1_input, audio2_input], outputs=similarity_output).launch()

### Question 2

**Goal**: The goal of speech separation is to estimate individual speaker signals from their mixture, where the source signals may be overlapped with each other entirely or partially.

**Tasks**: 

    1. Generate the LibriMix dataset by combining two speakers from the LibriSpeech dataset, focusing solely on the LibriSpeech test clean partition. 

    2. Partition the resulting LibriMix dataset into a 70-30 split for training and testing purposes. Evaluate the performance of the pre-trained SepFormer on the testing set, employing scale-invariant signal-to-noise ratio improvement (SISNRi) and signal-to-distortion ratio improvement (SDRi) as metrics. 

    3. Fine-tune the SepFormer model using the training set and report its performance on the test split of the LibriMix dataset.

    4. Provide observations on the changes in performance throughout the experiment.

### NOTE: Q2 Done in .py files